## Query Marketplace

This notebook constructs a "query marketplace", where our system selects the fingerprints with the highest utility to cost ratio at runtime and does lookups only on those fingerprints.

In [1]:
import pickle
import numpy as np
from ExtractBootlegScores import *
import itertools
import numba as nb
from numba import njit
from collections import defaultdict
import dill
from glob import iglob

**Load in databases and counts**

In [2]:
# this dictionary contains all the databases for each n-gram type included in our marketplace database
db_dir = "/data1/dyang/Marketplace_db/database/"

In [3]:
# this dictionary maps each n-gram type to its probability of correctness
probabilities_dir = "/data1/dyang/Marketplace_db/probabilities.pkl"

In [4]:
# this dictionary maps each number to the piece name (a string)
piece_mapping_dir = "/data1/dyang/Marketplace_db/num_to_piece_random.pkl"

In [5]:
d = {}

In [6]:
for filename in iglob(f"{db_dir}/*.pkl", recursive=True):
    combination = filename.split('/')[-1][:-4]
    with open(filename, "rb") as f:
        d[combination] = pickle.load(f)
    print(f"finished {combination}")

finished 015
finished 014
finished 035
finished 02
finished 03
finished 05
finished 013
finished 04
finished 023
finished 025
finished 045
finished 034
finished 01
finished 012
finished 0
finished 024


In [7]:
with open(probabilities_dir, "rb") as f:
    utilities = pickle.load(f)

In [8]:
with open(piece_mapping_dir, 'rb') as f:
    num_to_piece = pickle.load(f)

In [9]:
combinations = []
for n_gram in range(1, 4):
    combinations += [[0] + list(tup) for tup in itertools.combinations(range(1, 6), n_gram-1)]

In [10]:
combinations = ["".join(str(num) for num in combination) for combination in combinations]

In [11]:
powers = 1 << np.arange(62)[::-1]

In [12]:
def compute_fingerprint(cols):
    fp = []
    equals_Zero = True
    for column in cols:
        hashint = int(column.dot(powers))
        fp.append(hashint)
        if hashint != 0:
            equals_Zero = False
    if equals_Zero == True:
        return None
    return tuple(fp)

Our utility to cost ratio is defined as the probability of correctness (utility) divided by the number of matches (cost), because the number of matches equals the number of lookups we would have to do for that fingerprint.

In [13]:
def utility(combination, matches):
    return utilities[combination] / matches

In [14]:
def get_ratios(bscore_query, rindex_dict):
    """Inputs: an L x 62 bootleg score query and our dictionary, where
               rindex_dict[fp] = (count, {dictionary of pieces and offsets})
        Output: a 16 X L table where each element is a tuple of (utility:cost ratio, combination, n_gram)"""
    l = len(bscore_query)
    # ratios[i][j] is a pair of (ratio, combination, fingerprint)
    ratios = np.array([[(0, None, None) for _ in range(l)] for _ in range(16)])
    for j in range(l):
        # calculate utility to cost ratio for all 16 n-grams
        for idx, combination in enumerate(combinations):
            cols = []
            # we need at least enough fingerprints for all the indices in our combination
            try:
                for i in combination:
                    cols.append(bscore_query[j+int(i)])
            except IndexError:
                continue
            fp = compute_fingerprint(cols)
            if not fp or combination not in rindex_dict or fp not in rindex_dict[combination]:
                continue
            matches = rindex_dict[combination][fp][0]
            if matches == 0:
                continue
            ratios[idx][j] = (utility(combination, matches), combination, fp)
    return ratios

In [15]:
def update_offset_dict(offset_dict, pieces_and_offsets, i, num_lookups):
    """Input: a dictionary mapping pieces to offsets for a given n-gram and the number of lookups we can do.
       Output: an updated dictionary of offsets containing all the lookups we just did. This dictionary
               will be used in the histogram of offsets."""
    if num_lookups == 0:
        return
    for piece in pieces_and_offsets:
        if num_lookups <= len(pieces_and_offsets[piece]):
            offset = [j - i for j in pieces_and_offsets[piece][:num_lookups]]
        else:
            offset = [j - i for j in pieces_and_offsets[piece]]
        offset_dict[num_to_piece[piece]].extend(offset)
        num_lookups -= len(pieces_and_offsets[piece])
        if num_lookups <= 0:
            break

In [16]:
def get_fingerprints(bscore_query, rindex_dict, ratios, runtime_budget):
    """This takes in a bootleg score for a query as its input, and does a certain number of lookups for each
       column of the bootleg score in accordance with our runtime budget. It then returns the updated dictionary of
       offsets containing all the lookups performed for the query, which is then used to compute the histogram of 
       offsets."""
    l = len(bscore_query)
    aisle_budget = runtime_budget // l
    cur_budget = aisle_budget
    offset_dict = defaultdict(list)
    matches_processed = 0
    for i in range(l):
        fingerprints = []
        col = ratios[:, i]
        lookups = sorted(col, key = lambda x: x[0], reverse = True)
        for _, combination, n_gram in lookups:
            if not n_gram or cur_budget < 0:
                break
            matches, pieces_and_offsets = rindex_dict[combination][n_gram]
            if cur_budget - matches < 0:
                num_lookups = cur_budget
            else:
                num_lookups = matches
            update_offset_dict(offset_dict, pieces_and_offsets, i, num_lookups)
            cur_budget -= num_lookups
            matches_processed += num_lookups
        cur_budget += aisle_budget
    return offset_dict, matches_processed

In [17]:
def rankHistograms(offset_dict, bin_size=10):
    """This implements the histogram of offsets method for ranking the predicted pieces."""
    bin_size = 3
    pieceScores = []
    for key, h in offset_dict.items():
        if not h:
            continue
        maxh = max(h)
        minh = min(h)
        hist = np.zeros(int((maxh-minh)/bin_size)+2)
        for i in h:
            hist[int((i-minh)/bin_size)] += 1
        score = np.max(hist)
        pieceScores.append((key, score))
            
    pieceScores = sorted(pieceScores, key = lambda x:x[1], reverse=True)
    return pieceScores

In [42]:
def processSingleQuery(imagefile, rindex, runtime_budget, outfile = None):
    """Inputs: a file representing a query image, a reverse index dictionary mapping each n-gram to its 
               offsets in IMSLP, and a runtime budget for the query.
       Output: a sorted list of predicted pieces and their scores based on the histogram of offsets method."""
    profileStart = time.time()
    
    # Get Bootleg Score
    bscore_query = processQuery(imagefile)
    bscore_query = bscore_query.T
    
    searchStart = time.time()
    # Generate and rank histograms
    
    ratios = get_ratios(bscore_query, rindex)
    offset_dict, matches_processed = get_fingerprints(bscore_query, rindex, ratios, runtime_budget)
    pieceScores = rankHistograms(offset_dict)
    # Profile & save to file
    profileEnd = time.time()
    
    profileDur = profileEnd - profileStart
    print(matches_processed)
    print(profileDur)
    saveToFile(outfile, imagefile, pieceScores, profileDur, matches_processed)
    return pieceScores

In [43]:
def saveToFile(outfile, imagefile, pieceScores, profileDur, matches_processed):
    if outfile:
        with open(outfile, 'wb') as f:
            query = os.path.splitext(os.path.basename(imagefile))[0]
            pickle.dump((query,pieceScores, profileDur, matches_processed),f)

In [44]:
processSingleQuery('data/CameraDataset/queries/p1_q1.jpg', d, 1000, outfile = None)

Processing data/CameraDataset/queries/p1_q1.jpg
749
0.8467371463775635


[('p1', 13.0),
 ('dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_00470', 9.0),
 ('dGouin,_PierreNocturnes,_Op.9_112335', 8.0),
 ('dGouin,_PierreNocturnes,_Op.9_34915', 7.0),
 ('dGouin,_PierreNocturnes,_Op.9_00470', 6.0),
 ('dGouin,_PierreNocturnes,_Op.9_86550', 5.0),
 ('dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_86550', 4.0),
 ('dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_113996', 3.0),
 ('dGouin,_PierreNocturnes,_Op.9_34916', 3.0),
 ('dChopin,_Fr%C3%A9d%C3%A9ricNocturnes,_Op.9_34916', 3.0),
 ('dBeethoven,_Ludwig_vanPiano_Sonata_No.22,_Op.54_51749', 3.0),
 ('dGouin,_PierreNocturnes,_Op.9_113996', 3.0),
 ('dMerkel,_Gustav_Adolf4_Pieces,_Op.74_367408', 3.0),
 ('dHuber,_Hans10_Grosse_Et%C3%BCden,_Op.9_101631', 2.0),
 ('dCzerny,_CarlThe_Art_of_Finger_Dexterity,_Op.740_04065', 2.0),
 ('dLeschetizky,_TheodorSouvenirs_d%27Italie,_Op.39_502118', 2.0),
 ('dCzerny,_CarlRondino_No.1_on_%27Cara_attendimi%27,_Op.22_355929', 2.0),
 ('dSchumann,_RobertWaldszenen,_Op.82_271971', 2.0),
 ('dChopin,_Fr%C3%A9d

In [45]:
def processQuery_wrapper(queryfile, rindex, outdir, runtime_budget):
    # wrapper for running multiple jobs in parallel
    basename = os.path.splitext(os.path.basename(queryfile))[0] # e.g. p1_q1
    hyp_outfile = "{}/{}.hyp".format(outdir, basename)
    piece = basename.split('_')[0]
    # might change later to print to outfile
    return processSingleQuery(queryfile, rindex, runtime_budget, hyp_outfile)

Here we set the runtime budget to 65000, which means we can process at most 65000 matches per query.

In [56]:
runtime_budget = 1000

In [57]:
query_list = '../Marketplace/cfg_files/scanned.train' # list of query images
outdir = f'/home/dyang/experiments_final/train/scanned1k' # where to save hypothesis output files

In [58]:
# prep output directory
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# load reverse index. Recommend keeping load=False and loading it earlier.
load = False
if load:
    print("LOADING RINDEX")
    rindex1 = []
    with open(pickle_file, 'rb') as f:
        rindex1 = pickle.load(f)
    rindex_filter = rindex1

print("STARTING PROCESSING")
# number of cores to use
multiprocess = False
if multiprocess:
    n_cores = 30 #multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=n_cores)

inputs = []
with open(query_list, 'r') as f:
    for line in f:
        inputs.append((line.rstrip(), outdir))

if multiprocess:
    # process queries in parallel
    outputs = list(pool.starmap(processQuery_wrapper, inputs))
else:
    for i in inputs:
        processQuery_wrapper(i[0], d, i[1], runtime_budget)

STARTING PROCESSING
Processing data/ScannedDataset/p111/p111-1.jpg
880
1.161064624786377
Processing data/ScannedDataset/p111/p111-0.jpg
888
0.9722204208374023
Processing data/ScannedDataset/p141/p141-1.jpg
652
0.8689262866973877
Processing data/ScannedDataset/p141/p141-2.jpg
844
1.0336666107177734
Processing data/ScannedDataset/p141/p141-4.jpg
900
0.9183859825134277
Processing data/ScannedDataset/p141/p141-0.jpg
309
1.102367877960205
Processing data/ScannedDataset/p141/p141-3.jpg
968
0.7632803916931152
Processing data/ScannedDataset/p45/p45-1.jpg
960
0.545651912689209
Processing data/ScannedDataset/p45/p45-2.jpg
840
0.5218989849090576
Processing data/ScannedDataset/p45/p45-3.jpg
952
0.5320672988891602
Processing data/ScannedDataset/p45/p45-0.jpg
804
0.5523898601531982
Processing data/ScannedDataset/p181/p181-7.jpg
744
0.792820930480957
Processing data/ScannedDataset/p181/p181-0.jpg
744
0.5583536624908447
Processing data/ScannedDataset/p181/p181-5.jpg
945
0.8359284400939941
Processing d

936
0.7719602584838867
Processing data/ScannedDataset/p115/p115-0.jpg
924
0.6236939430236816
Processing data/ScannedDataset/p61/p61-2.jpg
808
0.8744995594024658
Processing data/ScannedDataset/p61/p61-3.jpg
965
0.9513888359069824
Processing data/ScannedDataset/p61/p61-1.jpg
550
0.8405623435974121
Processing data/ScannedDataset/p61/p61-9.jpg
840
1.1030232906341553
Processing data/ScannedDataset/p61/p61-10.jpg
978
1.193221092224121
Processing data/ScannedDataset/p61/p61-4.jpg
749
1.0386543273925781
Processing data/ScannedDataset/p61/p61-0.jpg
600
0.32968616485595703
Processing data/ScannedDataset/p61/p61-6.jpg
0
0.29577159881591797
Processing data/ScannedDataset/p61/p61-7.jpg
975
0.9938640594482422
Processing data/ScannedDataset/p61/p61-5.jpg
864
0.9051647186279297
Processing data/ScannedDataset/p61/p61-8.jpg
992
0.3426973819732666
Processing data/ScannedDataset/p121/p121-1.jpg
767
0.6042027473449707
Processing data/ScannedDataset/p121/p121-0.jpg
888
0.5374252796173096
Processing data/Sca